In [ ]:
import random
import time
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split 
from functools import reduce
import sklearn.preprocessing
from sklearn.metrics import classification_report 
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.layers import Dropout
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential

In [ ]:
# global variables
safe_action = []
safe_result = []
safe_save_money = []
safe_earning = []
safe_player_initial_sum = []
safe_player_final_sum = []
safe_dealer_first_card = []


random_action = []
random_result = []
random_save_money = []
random_earning = []
random_player_initial_sum = []
random_player_final_sum = []
random_dealer_first_card = []


safe_win_loss_record = []
random_win_loss_record = []

playing = True
deck_cards = []

In [ ]:
# Basic definition of the cards. Simple as it is
class deck_of_cards:
  def __init__(self, card_suits, card_characters, card_point_value):
    self.card_suits = card_suits
    self.card_characters = card_characters
    self.card_point_value = card_point_value

In [ ]:
# Add a hidden function for the dealer second card in here some how
# Maybe use a boolean for the hidden card
def display_cards(cards):
    display_top_wall = ""
    for card in cards:
        display_top_wall = display_top_wall + "  ______________  "
    print(display_top_wall)
 

    display_vertical_wall = ""
    for card in cards:
        display_vertical_wall = display_vertical_wall + " |              | "
    print(display_vertical_wall)
 
    display_card_character = ""
    for card in cards:
        # if it is a 10, remove one of the spacing
        if card.card_characters == '10':
            display_card_character = display_card_character + " |  {}          | ".format(card.card_characters)
        # if not a 10, keep the normal spacing
        else:
            display_card_character = display_card_character + " |  {}           | ".format(card.card_characters)  
    print(display_card_character)

    for i in range (1,4):
        display_vertical_wall = ""
        for card in cards:
            display_vertical_wall = display_vertical_wall + " |              | "
        print(display_vertical_wall)
 
    display_card_suit = ""
    for card in cards:
        display_card_suit = display_card_suit + " |      {}      | ".format(card.card_suits)
    print(display_card_suit)    

    for i in range (1,4):
        display_vertical_wall = ""
        for card in cards:
            display_vertical_wall = display_vertical_wall + " |              | "
        print(display_vertical_wall)

    display_card_character = ""
    for card in cards:
        # if it is a 10, remove one of the spacing
        if card.card_characters == '10':
            display_card_character = display_card_character + " |          {}  | ".format(card.card_characters)
        # if not a 10, keep the normal spacing
        else:
            display_card_character = display_card_character + " |           {}  | ".format(card.card_characters)  
    print(display_card_character)  
         
    display_bottom_wall = ""
    for card in cards:
        display_bottom_wall = display_bottom_wall + " |______________| "
    print(display_bottom_wall)       
  
  

In [ ]:
def blackjack_rules():
  print("Rules of blackjack for the players are as follows: ")
  print("Players must have a higher score than the dealer, but cannot exceed 21")
  print("Players can either choose to 'hit' to get another card or 'stand' to keep their score")
  print("If the player gets 21, the player automatically wins.")
  print("If the player exceeds 21, the player bust and automatically loses.")
  print("If the player has the same score as the dealer, the game is a tie.")
  print("Players must enter a bet before the blackjack game can start.")
  
  print("\nRules of blackjack for the dealers are as follows:")
  print("The dealer must hit until their score exceeds 16. If dealer score is over 16, the dealer must stand.")
  print("If the dealer gets 21, the dealer automatically wins.")
  print("If the dealer exceeds 21, the player bust and automatically loses.")
  print("If the dealer has the same score as the dealer, the game is a tie.")

In [ ]:
def shuffle():
  card_suits = ["Hearts", "Diamonds", "Clubs", "Spades"] # There is only 4 suits in a deck of cards
  suits_graphics = {"Hearts":"\u2661", "Diamonds": "\u2662", "Clubs": "\u2667", "Spades":"\u2664"} # Graphics of the suits looks better
  card_characters = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"] # All possible cards characters in a deck

  # Assigns the point value to the characters of the cards 
  card_point_value = {"A": 1, "2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7, "8": 8, "9": 9, "10": 10, "J": 10, "Q": 10, "K": 10}
  deck_cards = [] # Only using 4 decks of cards

  # Sets up our 2 deck of cards
  for i in range(1,5):
    for suit in card_suits:
      for card in card_characters:
        deck_cards.append(deck_of_cards(suits_graphics[suit], card, card_point_value[card])) # 4 deck = 208 cards
  print("\nReshuffling 4 decks of cards Please wait!\n")
  return deck_cards
#print(len(deck_cards)) # 1 deck = 52 cards, 2 deck = 104 cards, etc... 
#blackjack_rules()

In [ ]:
def total_check(hand):
  #total of cards in hand
  sum = 0
  #keeps track on the number of ace cards counting as 11
  ace_count = 0
  for card in hand:
    sum += card.card_point_value
    #changes an ace to count for 11 if doing so would not go over 21 
    if card.card_point_value == 1:
      if sum < 12:
        sum += 10
        ace_count += 1
    # changes an ace back to counting for 1 if the total is over 21 and an ace is being treated as 11
    if sum > 21 and ace_count > 0:
      sum -= 10
      ace_count -= 1
  return sum

In [ ]:
def blackjack_check(hand, hand_sum):
  if len(hand) == 2 and hand_sum == 21:
    return True
  else:
    return False

In [ ]:
def safe_move(deck, player_cards, bet):
  playing = True
  split = False
  sum = total_check(player_cards)
  while playing == True:
    if sum < 11 :
      #print("HIT")
      new_card = random.choice(deck)
      player_cards.append(new_card)
      deck.remove(new_card)
      sum = total_check(player_cards)
      #print("\nPlayer Cards:")
      #display_cards(player_cards)
      player_total = total_check(player_cards)
      #print("Player's Total:", player_total,)
      safe_action.append(1)
      if sum > 21:
        #print("Bust.")
        playing = False
        break
      if sum == 21:
        playing = False
        break
      break
    elif sum > 11:
      safe_action.append(2)
      #print("STAY")
      playing = False
      break
    elif sum == 11:
      #print("Double Down")
      bet = bet * 2
      new_card = random.choice(deck)
      player_cards.append(new_card)
      deck.remove(new_card)
      sum = total_check(player_cards)
      #print("\nPlayer Cards:")
      #display_cards(player_cards)
      player_total = total_check(player_cards)
      #print("Player's Total:", player_total)
      safe_action.append(3)
      if sum > 21:
        #print("Bust.")
        playing = False
      playing = False
      break
  return deck, player_cards, bet, sum

In [ ]:
def random_move(deck, player_cards, bet):
  playing = True
  split = False
  sum = total_check(player_cards)
  import random
  get_random_action = random.uniform(0,1)
  #print(get_random_action)
  while playing == True:
    # If action is 0.65 or greater, hit
    if get_random_action >= 0.65 :
      #print("HIT")
      new_card = random.choice(deck)
      player_cards.append(new_card)
      deck.remove(new_card)
      sum = total_check(player_cards)
      #print("\nPlayer Cards:")
      #display_cards(player_cards)
      player_total = total_check(player_cards)
      #print("Player's Total:", player_total,)
      random_action.append(1)
      if sum > 21:
        #print("Bust.")
        playing = False
        break
      if sum == 21:
        playing = False
        break
      break
    # if action is between 0.30 and 0.64, the action will stay
    elif get_random_action >= 0.30 and get_random_action < 0.65:
      random_action.append(2)
      #print("STAY")
      playing = False
      break
    # if action is below 0.29, the action will double down
    elif get_random_action < 0.30:
      #print("Double Down")
      bet = bet * 2
      new_card = random.choice(deck)
      player_cards.append(new_card)
      deck.remove(new_card)
      sum = total_check(player_cards)
      #print("\nPlayer Cards:")
      #display_cards(player_cards)
      player_total = total_check(player_cards)
      #print("Player's Total:", player_total)
      random_action.append(3)
      if sum > 21:
        #print("Bust.")
        playing = False
      playing = False
      break
  return deck, player_cards, bet, sum

In [ ]:
def player_move(deck, player_cards, bet):
  playing = True
  split = False
  sum = total_check(player_cards)
  while playing == True:
    choice = input("Enter: 1 to hit, 2 to stay, or 3 to double down: ")
    if choice == "1":
      new_card = random.choice(deck)
      player_cards.append(new_card)
      deck.remove(new_card)
      sum = total_check(player_cards)
      #print("\nPlayer Cards:")
      display_cards(player_cards)
      player_total = total_check(player_cards)
      #print("Player's Total:", player_total,"\n")
      if sum > 21:
        #print("Bust.")
        playing = False
        break
      if sum == 21:
        playing = False
        break
    elif choice == "2":
      playing = False
      break
    elif choice == "3":
      bet = bet * 2
      new_card = random.choice(deck)
      player_cards.append(new_card)
      deck.remove(new_card)
      sum = total_check(player_cards)
      #print("\nPlayer Cards:")
      display_cards(player_cards)
      player_total = total_check(player_cards)
      #print("Player's Total:", player_total,"\n")
      if sum > 21:
        #print("Bust.")
        playing = False
      playing = False
      break
  return deck, player_cards, bet, sum

In [ ]:
def dealer_move(deck, dealer_cards,computer_play):
  total = 0
  dealer_total = total_check(dealer_cards)
  while dealer_total < 17:
    new_card = random.choice(deck)
    dealer_cards.append(new_card)
    deck.remove(new_card)
    if(computer_play == False):
      #print("Dealer Cards:")
      display_cards(dealer_cards)
    ace_count = 0
    for card in dealer_cards:
      total += card.card_point_value
      #changes an ace to count for 11 if doing so would not go over 21 
      if card.card_point_value == 1:
        if total < 12:
          total += 10
          ace_count += 1
      # changes an ace back to counting for 1 if the total is over 21 and an ace is being treated as 11
      if total > 21 and ace_count > 0:
        total -= 10
        ace_count -= 1
    #After the cards in hand have been counted and the total is 17 and an ace counts as 11, make the ace count as 1 to keep adding cards
    if total == 17 and ace_count > 0:
      total -= 10
      ace_count -= 1
    sum = total_check(dealer_cards)
    dealer_total = total_check(dealer_cards)
    #print("\nDealer's Total:", dealer_total,"\n")
  return deck, dealer_cards, dealer_total



In [ ]:
def safe_game(player_money, deck_cards, computer_play):
  bet = 5
  # Deals the player a random card.
  player_cards = []
  dealer_cards = []
  #player_card = random.choice(deck_cards)
  #player_cards.append(player_card)
  #deck.remove(player_card)

  safe_play_win = []

  player_bust = False
  dealer_bust = False
  count = 0
  # Need to have this display both cards one at a time. Having issues with it atm
  while len(player_cards) < 2:
    # Deals the player a random card.
    deal_player_card = random.choice(deck_cards)
    player_cards.append(deal_player_card)
    deck_cards.remove(deal_player_card)
    # Deal the dealer a random card.
    deal_dealer_card = random.choice(deck_cards)
    dealer_cards.append(deal_dealer_card)
    deck_cards.remove(deal_dealer_card)
    # Gets the first card of the dealer.
    if count == 0:
      dealer_card_1 = total_check(dealer_cards)
      safe_dealer_first_card.append(dealer_card_1)
      count += 1

  if(computer_play == False):
    print("Dealer Cards:")
    display_cards(dealer_cards)

  dealer_total = total_check(dealer_cards)
  #print("\nDealer's Total:", dealer_total,"\n")

  if(computer_play == False):
    print("Player Cards:")
    display_cards(player_cards)

  player_total = total_check(player_cards)
  #print("Player's Total:", player_total)
  safe_player_initial_sum.append(player_total)

  player_blackjack = blackjack_check(player_cards, player_total)
  dealer_blackjack = blackjack_check(dealer_cards, dealer_total)

  if dealer_blackjack == True and player_blackjack == False:
    print("Blackjack! Dealer wins!")
    player_money -= bet
    safe_action.append(0)
    safe_result.append(0)

  elif dealer_blackjack == False and player_blackjack == True:
    print("Blackjack! Player wins!")
    player_money += bet * (3/2)
    safe_action.append(0)
    safe_result.append(1)

  elif dealer_blackjack == True and player_blackjack == True:
    print("Two blackjacks! Push!")
    safe_action.append(0)
    safe_result.append(2)

  else:
    if(computer_play == False):
      deck_cards, player_cards, bet, player_total = player_move(deck_cards, player_cards, bet)
      deck_cards, dealer_cards, dealer_total = dealer_move(deck_cards, dealer_cards)
    else:
      deck_cards, player_cards, bet,  player_total = safe_move(deck_cards, player_cards, bet)
      deck_cards, dealer_cards, dealer_total = dealer_move(deck_cards, dealer_cards, computer_play)


    if (player_total > 21):
      print("Player bust! Dealer wins! Player total:", player_total)
      player_bust = True
      player_money = player_money - bet
      safe_result.append(0)
      

    if (dealer_total > 21):
      print("Dealer bust! Player wins! Dealer total:", dealer_total)
      dealer_bust = True
      player_money = player_money + bet
      safe_result.append(1)
      

    if (player_bust == False and dealer_bust == False and player_total > dealer_total): 
      print("Player wins!")
      player_money += bet
      safe_result.append(1)

    elif (player_bust == False and dealer_bust == False and player_total < dealer_total):
      print("Dealer wins!")
      player_money -= bet
      safe_result.append(0)

    elif (player_bust == False and dealer_bust == False and player_total == dealer_total):
      print("It's a push!")
      safe_result.append(2)

  #print("THIS IS PLAYER TOTAL:", player_total)
  #print("THIS IS DEALER TOTAL:", dealer_total)
  safe_player_final_sum.append(player_total)

  card_number_check = len(deck_cards)
  #print("This is amount of cards left:", card_number_check)

  if card_number_check < 20:
    deck_cards = shuffle()
  return player_money, deck_cards

In [ ]:
def random_game(player_money, deck_cards, computer_play):
  bet = 5
  # Deals the player a random card.
  player_cards = []
  dealer_cards = []
  #player_card = random.choice(deck_cards)
  #player_cards.append(player_card)
  #deck.remove(player_card)

  random_play_win = []

  player_bust = False
  dealer_bust = False
  count = 0
  # Need to have this display both cards one at a time. Having issues with it atm
  while len(player_cards) < 2:
    # Deals the player a random card.
    deal_player_card = random.choice(deck_cards)
    player_cards.append(deal_player_card)
    deck_cards.remove(deal_player_card)
    # Deal the dealer a random card.
    deal_dealer_card = random.choice(deck_cards)
    dealer_cards.append(deal_dealer_card)
    deck_cards.remove(deal_dealer_card)
    # Gets the first card of the dealer.
    if count == 0:
      dealer_card_1 = total_check(dealer_cards)
      random_dealer_first_card.append(dealer_card_1)
      count += 1

  if(computer_play == False):
    print("Dealer Cards:")
    display_cards(dealer_cards)

  dealer_total = total_check(dealer_cards)
  #print("\nDealer's Total:", dealer_total,"\n")

  if(computer_play == False):
    print("Player Cards:")
    display_cards(player_cards)

  player_total = total_check(player_cards)
  #print("Player's Total:", player_total)
  random_player_initial_sum.append(player_total)

  player_blackjack = blackjack_check(player_cards, player_total)
  dealer_blackjack = blackjack_check(dealer_cards, dealer_total)

  if dealer_blackjack == True and player_blackjack == False:
    print("Blackjack! Dealer wins!")
    player_money -= bet
    random_action.append(0)
    random_result.append(0)

  elif dealer_blackjack == False and player_blackjack == True:
    print("Blackjack! Player wins!")
    player_money += bet * (3/2)
    random_action.append(0)
    random_result.append(1)

  elif dealer_blackjack == True and player_blackjack == True:
    print("Two blackjacks! Push!")
    random_action.append(0)
    random_result.append(2)

  else:
    if(computer_play == False):
      deck_cards, player_cards, bet, player_total = player_move(deck_cards, player_cards, bet)
      deck_cards, dealer_cards, dealer_total = dealer_move(deck_cards, dealer_cards)
    else:
      deck_cards, player_cards, bet,  player_total = random_move(deck_cards, player_cards, bet)
      deck_cards, dealer_cards, dealer_total = dealer_move(deck_cards, dealer_cards, computer_play)


    if (player_total > 21):
      print("Player bust! Dealer wins! Player total:", player_total)
      player_bust = True
      player_money = player_money - bet
      random_result.append(0)
      

    elif (dealer_total > 21):
      print("Dealer bust! Player wins! Dealer total:", dealer_total)
      dealer_bust = True
      player_money = player_money + bet
      random_result.append(1)
      

    if (player_bust == False and dealer_bust == False and player_total > dealer_total): 
      print("Player wins!")
      player_money += bet
      random_result.append(1)

    elif (player_bust == False and dealer_bust == False and player_total < dealer_total):
      print("Dealer wins!")
      player_money -= bet
      random_result.append(0)

    elif (player_bust == False and dealer_bust == False and player_total == dealer_total):
      print("It's a push!")
      random_result.append(2)

  #print("THIS IS PLAYER TOTAL:", player_total)
  #print("THIS IS DEALER TOTAL:", dealer_total)
  random_player_final_sum.append(player_total)

  card_number_check = len(deck_cards)
  #print("This is amount of cards left:", card_number_check)

  if card_number_check < 20:
    deck_cards = shuffle()
  return player_money, deck_cards

In [ ]:
blackjack_rules()

Rules of blackjack for the players are as follows: 
Players must have a higher score than the dealer, but cannot exceed 21
Players can either choose to 'hit' to get another card or 'stand' to keep their score
If the player gets 21, the player automatically wins.
If the player exceeds 21, the player bust and automatically loses.
If the player has the same score as the dealer, the game is a tie.
Players must enter a bet before the blackjack game can start.

Rules of blackjack for the dealers are as follows:
The dealer must hit until their score exceeds 16. If dealer score is over 16, the dealer must stand.
If the dealer gets 21, the dealer automatically wins.
If the dealer exceeds 21, the player bust and automatically loses.
If the dealer has the same score as the dealer, the game is a tie.


In [ ]:
computer_play = True
safe_play_money = 75000
update_money = 0
safe_profit_or_loss = 0
int(safe_play_money)
deck_cards = shuffle()
for i in range(0,40000,1):
  if(safe_play_money < 5):
    break
  if(safe_play_money > safe_play_money*2):
    break
  print("Game round:",i)
  update_money, deck_cards = safe_game(safe_play_money, deck_cards, computer_play)
  #print("\nPrevious $", safe_play_money)
  #print("Player $", update_money)
  safe_profit_or_loss = update_money - safe_play_money
  safe_play_money = update_money
  safe_save_money.append(update_money)
  safe_earning.append(safe_profit_or_loss)
  #print("This is profit or loss:",safe_profit_or_loss)
  

Streaming output truncated to the last 5000 lines.
Game round: 37596
Dealer bust! Player wins! Dealer total: 24
Game round: 37597
It's a push!
Game round: 37598
Dealer wins!
Game round: 37599
Dealer wins!
Game round: 37600
Blackjack! Player wins!
Game round: 37601
Dealer wins!
Game round: 37602
Dealer bust! Player wins! Dealer total: 22
Game round: 37603
Dealer wins!
Game round: 37604
It's a push!
Game round: 37605
Dealer wins!
Game round: 37606
Dealer bust! Player wins! Dealer total: 23
Game round: 37607
Blackjack! Player wins!
Game round: 37608
Player wins!
Game round: 37609
It's a push!
Game round: 37610
Dealer wins!
Game round: 37611
Dealer bust! Player wins! Dealer total: 22
Game round: 37612
Dealer wins!
Game round: 37613
Blackjack! Player wins!
Game round: 37614
Dealer bust! Player wins! Dealer total: 22
Game round: 37615
Dealer wins!
Game round: 37616
Dealer bust! Player wins! Dealer total: 24
Game round: 37617
Blackjack! Dealer wins!
Game round: 37618
Dealer wins!
Game round: 

In [ ]:
computer_play = True
random_play_money = 75000
random_update_money = 0
random_profit_or_loss = 0
int(random_play_money)
deck_cards = shuffle()
for i in range(0,40000,1):
  if(random_play_money < 5):
    break
  if(random_play_money > random_play_money*2):
    break
  print("Game round:",i)
  random_update_money, deck_cards = random_game(random_play_money, deck_cards, computer_play)
  #print("\nPrevious $", random_play_money)
  #print("Player $", random_update_money)
  random_profit_or_loss = random_update_money - random_play_money
  random_play_money = random_update_money
  random_save_money.append(random_update_money)
  random_earning.append(random_profit_or_loss)
  #print("This is profit or loss:",random_profit_or_loss)

Streaming output truncated to the last 5000 lines.
Game round: 37602
Dealer wins!
Game round: 37603
Dealer bust! Player wins! Dealer total: 23
Game round: 37604
Dealer wins!
Game round: 37605
Dealer wins!
Game round: 37606
Dealer wins!
Game round: 37607
Dealer bust! Player wins! Dealer total: 25
Game round: 37608
Dealer wins!
Game round: 37609
Player bust! Dealer wins! Player total: 30
Game round: 37610
Dealer wins!
Game round: 37611
Blackjack! Dealer wins!
Game round: 37612
Dealer bust! Player wins! Dealer total: 24

Reshuffling 4 decks of cards Please wait!

Game round: 37613
Player bust! Dealer wins! Player total: 24
Game round: 37614
Dealer wins!
Game round: 37615
Dealer wins!
Game round: 37616
Player wins!
Game round: 37617
Dealer wins!
Game round: 37618
It's a push!
Game round: 37619
Player bust! Dealer wins! Player total: 22
Game round: 37620
Blackjack! Player wins!
Game round: 37621
Dealer bust! Player wins! Dealer total: 26
Game round: 37622
Player bust! Dealer wins! Player to

In [ ]:
safe_dataframe = pd.DataFrame()
safe_dataframe['Player Initial'] = safe_player_initial_sum
safe_dataframe['Dealer Hand'] = safe_dealer_first_card
safe_dataframe['Action'] = safe_action
safe_dataframe['Earning'] = safe_earning
safe_dataframe['Money'] = safe_save_money
safe_dataframe['Result'] = safe_result

In [ ]:
# Player Initial is player starting sum 
# Dealer hand is the first card they get. 
# Action varies. (0: no action, 1: hit, 2: stay, 3: double down)
# Earning varies. Anything negative is a loss, 0's is a push, and anything + is a win
# Money Keeps count of the money. They start with $1000 
# Result varies. (0: lose, 1: win, 2: tie)
safe_dataframe

,Player Initial,Dealer Hand,Action,Earning,Money,Result
0,11,3,3,10.0,75010.0,1
1,12,3,2,5.0,75015.0,1
2,14,8,2,5.0,75020.0,1
3,19,3,2,5.0,75025.0,1
4,10,11,0,-5.0,75020.0,0
...,...,...,...,...,...,...
39995,21,10,0,0.0,58087.5,2
39996,12,7,2,5.0,58092.5,1
39997,10,5,1,5.0,58097.5,1
39998,16,10,2,5.0,58102.5,1


In [ ]:
safe_win = 0
safe_lose = 0
safe_tie = 0
get_safe_game_played = len(safe_result)
for i in range(0,get_safe_game_played):
  if(safe_result[i] == 0):
    safe_lose += 1
  elif(safe_result[i] == 1):
    safe_win += 1
  elif(safe_result[i] == 2):
    safe_tie += 1

round_win = round((safe_win / get_safe_game_played),2) * 100
round_lose = round((safe_lose / get_safe_game_played),2) * 100
round_tie = round((safe_tie / get_safe_game_played),2) * 100

print("Statistics using random action strategy")
print("Win percentage:", str(round_win) + "%")
print("Lose percentage:", str(round_lose) + "%")
print("Tie percentage:", str(round_tie) + "%")

Statistics using random action strategy
Win percentage: 41.0%
Lose percentage: 53.0%
Tie percentage: 6.0%


In [ ]:
random_dataframe = pd.DataFrame()
random_dataframe['Player Initial'] = random_player_initial_sum
random_dataframe['Dealer Hand'] = random_dealer_first_card
random_dataframe['Action'] = random_action
random_dataframe['Earning'] = random_earning
random_dataframe['Money'] = random_save_money
random_dataframe['Result'] = random_result

In [ ]:
# Player Initial is player starting sum 
# Dealer hand is the first card they get. 
# Action varies. (0: no action, 1: hit, 2: stay, 3: double down)
# Earning varies. Anything negative is a loss, 0's is a push, and anything + is a win
# Money Keeps count of the money. They start with $1000 
# Result varies. (0: lose, 1: win, 2: tie)
random_dataframe

,Player Initial,Dealer Hand,Action,Earning,Money,Result
0,15,4,1,5.0,75005.0,1
1,17,8,3,10.0,75015.0,1
2,16,10,3,-10.0,75005.0,0
3,17,8,1,5.0,75010.0,1
4,18,8,2,-5.0,75005.0,0
...,...,...,...,...,...,...
39995,9,4,3,-10.0,2540.0,0
39996,11,9,3,10.0,2550.0,1
39997,16,4,1,-5.0,2545.0,0
39998,14,11,1,-5.0,2540.0,0


In [ ]:
random_win = 0
random_lose = 0
random_tie = 0
get_random_game_played = len(random_result)
for i in range(0,get_random_game_played):
  if(random_result[i] == 0):
    random_lose += 1
  elif(random_result[i] == 1):
    random_win += 1
  elif(random_result[i] == 2):
    random_tie += 1

round_win = round((random_win / get_random_game_played),2) * 100
round_lose = round((random_lose / get_random_game_played),2) * 100
round_tie = round((random_tie / get_random_game_played),2) * 100

print("Statistics using random action strategy")
print("Win percentage:", str(round_win) + "%")
print("Lose percentage:", str(round_lose) + "%")
print("Tie percentage:", str(round_tie) + "%")

Statistics using random action strategy
Win percentage: 34.0%
Lose percentage: 62.0%
Tie percentage: 5.0%


In [ ]:
# Merging both the safe_dataframe and random_dataframe
both_gameplay_dataset = [safe_dataframe, random_dataframe]
merge_dataframe = pd.concat(both_gameplay_dataset)
merge_dataframe

,Player Initial,Dealer Hand,Action,Earning,Money,Result
0,11,3,3,10.0,75010.0,1
1,12,3,2,5.0,75015.0,1
2,14,8,2,5.0,75020.0,1
3,19,3,2,5.0,75025.0,1
4,10,11,0,-5.0,75020.0,0
...,...,...,...,...,...,...
39995,9,4,3,-10.0,2540.0,0
39996,11,9,3,10.0,2550.0,1
39997,16,4,1,-5.0,2545.0,0
39998,14,11,1,-5.0,2540.0,0


In [ ]:
# Remove our Results and money. This will help train 
Training_Set = merge_dataframe.drop(['Result','Money'], axis = 1) 
Training_Set

,Player Initial,Dealer Hand,Action,Earning
0,11,3,3,10.0
1,12,3,2,5.0
2,14,8,2,5.0
3,19,3,2,5.0
4,10,11,0,-5.0
...,...,...,...,...
39995,9,4,3,-10.0
39996,11,9,3,10.0
39997,16,4,1,-5.0
39998,14,11,1,-5.0


In [ ]:
Result_Set = merge_dataframe['Result']
Result_Set

0        1
1        1
2        1
3        1
4        0
        ..
39995    0
39996    1
39997    0
39998    0
39999    0
Name: Result, Length: 80000, dtype: int64

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(Training_Set, Result_Set, test_size = 0.20, random_state = 37)
x_train # This will display the 80% of the randomized rows that will be used to train the ANN

,Player Initial,Dealer Hand,Action,Earning
8587,15,2,1,5.0
25077,13,6,2,-5.0
12083,17,10,3,-10.0
39266,12,11,0,-5.0
32584,17,11,0,-5.0
...,...,...,...,...
26524,12,8,2,-5.0
30971,20,11,0,-5.0
34572,12,10,0,-5.0
20843,12,2,2,-5.0


In [ ]:
x_test

,Player Initial,Dealer Hand,Action,Earning
20666,21,4,0,7.5
22760,15,7,2,5.0
3182,4,10,2,-5.0
19980,20,6,2,5.0
27782,13,10,2,-5.0
...,...,...,...,...
21937,14,11,2,-5.0
16390,15,9,2,-5.0
25528,20,10,2,5.0
34343,14,11,0,-5.0


In [ ]:
scaler = MinMaxScaler()
# Transforming the data for x_train using MinMaxScaler
x_train_scaler = scaler.fit_transform(x_train)
x_train_scaler 

array([[0.64705882, 0.        , 0.33333333, 0.75      ],
       [0.52941176, 0.44444444, 0.66666667, 0.25      ],
       [0.76470588, 0.88888889, 1.        , 0.        ],
       ...,
       [0.47058824, 0.88888889, 0.        , 0.25      ],
       [0.47058824, 0.        , 0.66666667, 0.25      ],
       [0.23529412, 0.88888889, 0.33333333, 0.25      ]])

In [ ]:
scaler = MinMaxScaler()
# Transforming the data for x_test using MinMaxScaler
x_test_scaler = scaler.fit_transform(x_test)
x_test_scaler

array([[1.        , 0.22222222, 0.        , 0.875     ],
       [0.64705882, 0.55555556, 0.66666667, 0.75      ],
       [0.        , 0.88888889, 0.66666667, 0.25      ],
       ...,
       [0.94117647, 0.88888889, 0.66666667, 0.75      ],
       [0.58823529, 1.        , 0.        , 0.25      ],
       [0.70588235, 0.88888889, 0.        , 0.25      ]])

In [ ]:
# Now we will take the data from our safe_play and our random_play to train our model. 
blackjack_AI = Sequential()
blackjack_AI.add(Dense(50))
blackjack_AI.add(Dense(40))
blackjack_AI.add(Dense(30))
blackjack_AI.add(Dense(20))
blackjack_AI.add(Dense(10))
blackjack_AI.add(Dense(7))
blackjack_AI.add(Dense(5))
blackjack_AI.add(Dense(3))
blackjack_AI.add(Dense(1, activation='sigmoid')) # The output neurons are subject to change depending on number of classes found.

blackjack_AI.compile(optimizer = 'adam', loss="categorical_crossentropy", metrics=["accuracy"])
blackjack_AI.fit(x = x_train_scaler, y = y_train, batch_size = 25, validation_split = 0.20, epochs = 20)

Epoch 1/20
25600/25600 [==============================] - 49s 2ms/step - loss: nan - accuracy: 0.5735 - val_loss: nan - val_accuracy: 0.5773
Epoch 2/20
25600/25600 [==============================] - 49s 2ms/step - loss: nan - accuracy: 0.5739 - val_loss: nan - val_accuracy: 0.5773
Epoch 3/20
25600/25600 [==============================] - 50s 2ms/step - loss: nan - accuracy: 0.5651 - val_loss: nan - val_accuracy: 0.5773
Epoch 4/20
25600/25600 [==============================] - 49s 2ms/step - loss: nan - accuracy: 0.5710 - val_loss: nan - val_accuracy: 0.5773
Epoch 5/20
25600/25600 [==============================] - 49s 2ms/step - loss: nan - accuracy: 0.5744 - val_loss: nan - val_accuracy: 0.5773
Epoch 6/20
25600/25600 [==============================] - 48s 2ms/step - loss: nan - accuracy: 0.5739 - val_loss: nan - val_accuracy: 0.5773
Epoch 7/20
25600/25600 [==============================] - 49s 2ms/step - loss: nan - accuracy: 0.5713 - val_loss: nan - val_accuracy: 0.5773
Epoch 8/20
25

In [ ]:
blackjack_AI.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (2, 50)                   250       
_________________________________________________________________
dense_37 (Dense)             (2, 40)                   2040      
_________________________________________________________________
dense_38 (Dense)             (2, 30)                   1230      
_________________________________________________________________
dense_39 (Dense)             (2, 20)                   620       
_________________________________________________________________
dense_40 (Dense)             (2, 10)                   210       
_________________________________________________________________
dense_41 (Dense)             (2, 7)                    77        
_________________________________________________________________
dense_42 (Dense)             (2, 5)                   